# 01 - Basic EAP Walkthrough
This notebook demonstrates running simplified EAP on a few toy IOI examples.

In [ ]:
# Ensure the repo root (containing `src/` and `data/`) is on sys.path
import sys, os
from pathlib import Path

def add_repo_root(marker_dir='src'):
    path = os.path.abspath(os.getcwd())
    prev = None
    while path and path != prev:
        if os.path.isdir(os.path.join(path, marker_dir)):
            if path not in sys.path:
                sys.path.insert(0, path)
            return path
        prev = path
        path = os.path.dirname(path)
    # fallback: parent of current working dir
    nb_dir = os.path.abspath(os.getcwd())
    candidate = os.path.abspath(os.path.join(nb_dir, '..'))
    if os.path.isdir(os.path.join(candidate, marker_dir)):
        if candidate not in sys.path:
            sys.path.insert(0, candidate)
        return candidate
    return os.path.abspath(os.getcwd())

REPO_ROOT = add_repo_root()
DATA_PATH = str(Path(REPO_ROOT) / 'data' / 'ioi_examples.json')


In [ ]:
import torch
from transformer_lens import HookedTransformer

from src.ioi_task import load_ioi_examples
from src.circuit_finder import run_eap_on_ioi, evaluate_ablation_plan
from src.visualization import plot_head_importance, plot_top_heads_graph

model = HookedTransformer.from_pretrained('gpt2-small')
examples = load_ioi_examples(DATA_PATH)
clean = []
corr = []
for ex in examples[:8]:
    clean.append(model.to_tokens(ex.clean)[0])
    corr.append(model.to_tokens(ex.corrupted)[0])
clean = torch.nn.utils.rnn.pad_sequence(clean, batch_first=True, padding_value=model.tokenizer.eos_token_id)
corr = torch.nn.utils.rnn.pad_sequence(corr, batch_first=True, padding_value=model.tokenizer.eos_token_id)

# Run EAP
res = run_eap_on_ioi(model, clean, corr, target_token_idx=-1)

# Save plots under repo root
save_dir = Path(REPO_ROOT) / 'results' / 'plots'
save_dir.mkdir(parents=True, exist_ok=True)
plot_head_importance(res.head_importance, save_path=str(save_dir / 'head_importance_basic.png'))
plot_top_heads_graph(res.head_importance, save_path=str(save_dir / 'top_heads_basic.png'))
res.head_importance.max(), res.head_importance.mean(), res.head_importance.shape


In [ ]:
# Validate causality via ablation of top-k heads
import numpy as np

imp = res.head_importance
layers, heads = imp.shape
flat = imp.flatten()
top_vals, top_pos = torch.topk(flat, k=min(5, flat.numel()))
top_heads = [(int(p)//heads, int(p)%heads) for p in top_pos]

metrics_top5 = evaluate_ablation_plan(
    model, clean, corr, target_token_idx=-1, heads_to_ablate=top_heads
)

# Random control with same count
rng = torch.Generator().manual_seed(0)
rand_pos = torch.randperm(flat.numel(), generator=rng)[:len(top_heads)]
rand_heads = [(int(p)//heads, int(p)%heads) for p in rand_pos]
metrics_rand = evaluate_ablation_plan(
    model, clean, corr, target_token_idx=-1, heads_to_ablate=rand_heads
)
metrics_top5, metrics_rand, top_heads
